In [2]:
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from typing import TypedDict, Literal

# 1️⃣ Define the state — this will hold the fish data and intermediate results

class FishState(TypedDict):
    description: str
    classification: str
    is_endangered: bool
    tag_needed: bool

# 2️⃣ Set up the model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3️⃣ Nodes (steps in the graph)

# ➤ Classify the fish
def classify_fish(state: FishState) -> FishState:
    response = llm([
        SystemMessage(content="You are a marine biologist."),
        HumanMessage(content=f"What kind of fish is this: {state['description']}?")
        ])
    return {**state, "classification": response.content.strip()}

# ➤ Check if the fish is endangered
def check_endangered(state: FishState) -> FishState:
    response = llm([
        SystemMessage(content="You are a marine conservation expert."),
        HumanMessage(content=f"Is the {state['classification']} endangered? Answer true or false.")
    ])
    is_endangered = response.content.strip().lower() == "true"
    return {**state, "is_endangered": is_endangered}

# ➤ Decide whether to tag
def decide_tagging(state: FishState) -> FishState:
    if state['is_endangered']:
        tag_needed = True
    else:
        response = llm([
            SystemMessage(content="You are a fish tagging specialist."),
            HumanMessage(content=f"Should we tag the {state['classification']}? Answer true or false.")
        ])
        tag_needed = response.content.strip().lower() == "true"
    return {**state, "tag_needed": tag_needed}

# 4️⃣ Build the graph

builder = StateGraph(FishState)

builder.add_node("classify_fish", classify_fish)
builder.add_node("check_endangered", check_endangered)
builder.add_node("decide_tagging", decide_tagging)

builder.set_entry_point("classify_fish")
builder.add_edge("classify_fish", "check_endangered")
builder.add_edge("check_endangered", "decide_tagging")
builder.add_edge("decide_tagging", END)

# 5️⃣ Create the graph

graph = builder.compile()

input_state = {"description": "A long, silver fish with an extended jaw and sharp teeth."}
final_result = graph.invoke(input_state)

print(final_result)




{'description': 'A long, silver fish with an extended jaw and sharp teeth.', 'classification': 'Based on your description of a long, silver fish with an extended jaw and sharp teeth, it sounds like you might be referring to a species of barracuda. Barracudas are known for their elongated bodies, sharp teeth, and predatory nature. They are typically found in warmer waters and are recognized for their speed and agility while hunting. Another possibility could be the needlefish, which also has a long, slender body and sharp teeth, but they are generally less robust than barracudas. If you have more specific details or a picture, I could help narrow it down further!', 'is_endangered': False, 'tag_needed': False}
